In [15]:
import pandas as pd
import geopandas as gp
import numpy as np
import rasterstats

Buildings processing
- create raster topo from contours
- subtract out contours from building heights to extrude in rhino

In [3]:
extent_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/SP clipping area.shp"
extent = gp.read_file(extent_path)

In [4]:
# set watershed boundary
watershed_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/green_duwamish_watershed.shp"
watershed = gp.read_file(watershed_path)

In [5]:
# find intersection of watershed boundary and extent
watershed_extent_intersect = gp.overlay(watershed, extent, how='intersection')

In [6]:
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

In [7]:
buildings['centroid_geom'] = buildings.centroid
buildings.set_geometry('centroid_geom',inplace = True)


In [8]:
buildings_clip = gp.overlay(buildings, watershed_extent_intersect, how = "intersection")

In [9]:
buildings_merge = pd.merge(buildings_clip, buildings[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')

In [10]:
buildings_merge.columns

Index(['OBJECTID', 'BP99_TYPE', 'BP99_APEX', 'BP99_BLDGK', 'FEATURE_ID',
       'YEARUPDATE', 'SHAPE_Leng', 'SHAPE_Area', 'geometry_x', 'AreaAcres',
       'AreaSqKm', 'States', 'HUC12', 'Name', 'HUType', 'HUMod', 'ToHUC',
       'TNMID', 'LoadDate', 'NonContrib', 'NonContr_1', 'GNIS_ID',
       'MetaSource', 'GlobalID', 'Shape__Are', 'Shape__Len', 'id',
       'geometry_y'],
      dtype='object')

In [12]:
buildings_merge.set_geometry('geometry_y', inplace = True)

In [14]:
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','geometry_y']].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Clipped/test.shp")

In [17]:
## used topo to raster tool with 5ft contours and 5 ft pixel resolution
# there may be better ways to get DEM / elevation values as raster
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/south_seattle_dem.tif"

In [24]:
zonal_stats = rasterstats.zonal_stats(buildings_merge, topo_rast, stats="count min mean max median std")

In [25]:
stats_df = pd.DataFrame.from_dict(zonal_stats).reset_index()
buildings_merge.reset_index(inplace = True)

In [26]:
merged_heights = pd.merge(buildings_merge, stats_df, how = 'left', left_on = 'index', right_on = 'index')

In [27]:
merged_heights

,level_0,index,OBJECTID,BP99_TYPE,BP99_APEX,BP99_BLDGK,FEATURE_ID,YEARUPDATE,SHAPE_Leng,SHAPE_Area,...,Shape__Are,Shape__Len,id,geometry_y,min,max,mean,count,std,median
0,0,0,263015,BLD,81.0,1603149.0,96359.0,NaN,137.581704,954.986190,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1269728.474 201505.377 0.000, 1269...",0.0,0.0,0.0,10,0.0,0.0
1,1,1,263016,BLD,66.0,1633032.0,332792.0,NaN,164.827831,1059.250511,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1269626.389 201657.008 0.000, 1269...",0.0,0.0,0.0,9,0.0,0.0
2,2,2,263017,BLD,67.0,1633034.0,254322.0,NaN,173.643943,1212.567710,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1269706.538 201640.549 0.000, 1269...",0.0,0.0,0.0,13,0.0,0.0
3,3,3,263018,BLD,64.0,1603150.0,96373.0,NaN,126.707617,799.976889,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1269806.989 201513.120 0.000, 1269...",0.0,0.0,0.0,9,0.0,0.0
4,4,4,263019,BLD,333.0,1370925.0,174844.0,NaN,155.019904,969.706351,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1277365.344 205658.228 0.000, 1277...",0.0,0.0,0.0,10,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18447,18447,18447,526892,BLD,85.0,0.0,529445.0,2015,279.660361,4450.874551,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1271490.789 192384.533 0.000, 1271...",0.0,0.0,0.0,44,0.0,0.0
18448,18448,18448,526948,BLD,42.0,0.0,504530.0,2015,487.845238,13246.238895,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1271810.141 198267.562 0.000, 1271...",0.0,0.0,0.0,136,0.0,0.0
18449,18449,18449,526949,BLD,55.0,0.0,504530.0,2015,748.231186,25748.409061,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1271555.667 198178.787 0.000, 1271...",0.0,0.0,0.0,257,0.0,0.0
18450,18450,18450,526950,BLD,38.0,0.0,504530.0,2015,360.422900,7628.829111,...,1.721548e+09,305066.764574,NaN,"POLYGON Z ((1271491.239 198365.218 0.000, 1271...",0.0,0.0,0.0,80,0.0,0.0


In [21]:
merged_heights['height_upd'] = (merged_heights['BP99_APEX']-merged_heights['mean']).astype(int)
merged_heights[['BP99_APEX','mean','min','max','median','std','height_upd']]

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer